In [ ]:
import pandas as pd 
import numpy as np
import datetime
from pathlib import Path
import os

pd.set_option('display.max_columns', 500)


In [ ]:
data_path = Path(os.getcwd()) / 'data' / 'WEC2022_Data'

In [ ]:
data_path

In [ ]:
stay_lenght_map = {-9999: None}
df['STAY_LENGTH_D'] = df['STAY_LENGTH_D'].replace(stay_lenght_map)
df['BOOKING_ARRIVAL_TIME_UTC'] = pd.to_datetime(df['BOOKING_ARRIVAL_TIME_UTC'])
df['BOOKING_DEPARTURE_TIME_UTC'] = pd.to_datetime(df['BOOKING_DEPARTURE_TIME_UTC'])
df['flight_len'] = df['BOOKING_ARRIVAL_TIME_UTC'] - df['BOOKING_DEPARTURE_TIME_UTC']
df['flight_len'] = df['flight_len'].apply(lambda x: x.seconds / 3600)

In [ ]:
bkg_drop_cols = ['UPGRADED_FLAG', 'UPGRADE_TYPE', 'UPGRADE_SALES_DATE']
bkg = pd.read_csv(data_path / 'BKG_train.csv', sep=';').drop(columns=bkg_drop_cols)

In [ ]:
df = fcp.merge(pd.read_csv(data_path / 'TKT_train.csv', sep=';'), on='TICKET_NUMBER', how='left')
df.shape

In [ ]:
df = df.merge(pd.read_csv(data_path / 'BKG_train.csv', sep=';').drop(columns=bkg_drop_cols), on='BOOKING_ID', how='left')
df.shape

In [ ]:
# df = df.merge(emd, how='left', left_on='TICKET_NUMBER', right_on='REFERENCE_TICKET_NUMBER')
# df.shape

In [ ]:
df.shape

In [ ]:
df.loc[:, [col for col in df.columns if 'DATE' in col]]

In [ ]:
(df[df['if_additional_upgrade'] == 1]['ORIGIN_AIRPORT_CODE'].value_counts() / df['ORIGIN_AIRPORT_CODE'].value_counts()).sort_values(ascending=False).head(30)

In [ ]:
# sus_air_type = (df[df['UPGRADED_FLAG'] == 'Y']['AIRCRAFT_TYPE'].value_counts() / df['AIRCRAFT_TYPE'].value_counts()).sort_values(ascending=False).head(5).index.tolist()

In [ ]:
sus_air_type = ['763', '788', '789', '332', '787']

In [ ]:
# sus_currency = (df[df['UPGRADED_FLAG'] == 1]['CURRENCY'].value_counts() / df['CURRENCY'].value_counts()).sort_values(ascending=False).head(6).index.tolist()

In [ ]:
sus_currency = ['JPY', 'USD', 'CAD', 'SGD', 'VND', 'AED']

In [ ]:
df

In [ ]:
df[[col for col in df.columns if col not in drop_cols]]

# Co zmieniamy 
1. 

In [ ]:
drop_cols = ['TICKET_NUMBER', 'UPGRADE_SALES_DATE', 'ORIGIN_AIRPORT_CODE', 'DESTINATION_AIRPORT_CODE', 'SALES_DATE', 'FLIGHT_DATE_LOCAL', 'MARKETING_CARRIER', 'OPERATIONAL_CARRIER', 
'BOOKED_CLASS', 'AIRCRAFT_TYPE', 'FARE_BASIS', 'UPGRADE_TYPE', 'UPGRADE_SALES_DATE', 'BOOKING_ID', 'ORIGINAL_TICKET_NUMBER', 'SEGMENTS', 'FLIGHT_COUPONS', 'FORM_OF_PAYMENT', 'CURRENCY', 'TOTAL_PRICE', 'LOYAL_CUSTOMER_ID', 
'LOYAL_CUSTOMER_DATE_OF_BIRTH', 'LOYAL_CUSTOMER_REGISTERED_DATE', 'SALES_DATE', 'SALES_MARKET', 'SEGMENTS', 'INTINERARY', 'BOOKING_ORIGIN_AIRPORT', 'BOOKING_ORIGIN_COUNTRY_CODE', 'BOOKING_DEPARTURE_TIME_UTC', 
'BOOKING_DESTINATION_AIRPORT', 'BOOKING_DESTINATION_COUNTRY_CODE', 'BOOKING_ARRIVAL_TIME_UTC']
oh_cols = ['FLIGHT_RANGE', 'BOOKED_CABIN', 'VAB', 'PAX_TYPE', 'SALES_CHANNEL', 'TRIP_TYPE', ]

In [ ]:
df['FLIGHT_DATE_LOCAL'] = pd.to_datetime(df['FLIGHT_DATE_LOCAL'])
df['SALES_DATE'] = pd.to_datetime(df['SALES_DATE'])
df['sale_to_flight_time'] = df['FLIGHT_DATE_LOCAL'] - df['SALES_DATE']
df['sale_to_flight_time'] = df['sale_to_flight_time'].apply(lambda x: x.days)

In [ ]:
df['TIME_DEPARTURE_LOCAL_TIME'] = pd.to_datetime(df['TIME_DEPARTURE_LOCAL_TIME'])
df['TIME_DEPARTURE_LOCAL_TIME'] = df['TIME_DEPARTURE_LOCAL_TIME'].apply(lambda x: x.hour)

In [ ]:
df['if_additional_upgrade'] = np.where(np.isin(df['TICKET_NUMBER'], emd['REFERENCE_TICKET_NUMBER'].unique()), 1, 0)

In [ ]:
df['same_carrier'] = np.where(df['MARKETING_CARRIER'] == df['OPERATIONAL_CARRIER'], 1, 0)

In [ ]:
df['is_sus_aircraft'] = np.where(np.isin(df['AIRCRAFT_TYPE'], sus_air_type), 1, 0)

In [ ]:
df['UPGRADED_FLAG'] = df['UPGRADED_FLAG'].map({'Y': 1, 'N': 0})

In [ ]:
sus_payment = (df[df['UPGRADED_FLAG'] == 1]['FORM_OF_PAYMENT'].value_counts() / df['FORM_OF_PAYMENT'].value_counts()).sort_values(ascending=False).head(9).index.tolist()

In [ ]:
df['is_sus_payment'] = np.where(np.isin(df['FORM_OF_PAYMENT'], sus_payment), 1, 0)

In [ ]:
df['intinerary_len'] = df['INTINERARY'].apply(lambda x: len(x.split('-')))

In [ ]:
df['PAX_GENDER'] = df['PAX_GENDER'].map({'M': 1, 'F': 0})

In [ ]:
df['CORPORATE_CONTRACT_FLG'] = df['CORPORATE_CONTRACT_FLG'].map({'Y': 1, 'N': 0})
df['LOYAL_CUSTOMER'] = df['LOYAL_CUSTOMER'].map({'Y': 1, 'N': 0})

In [ ]:
df['BOOKING_LONG_HOUL_FLAG'] = df['BOOKING_LONG_HOUL_FLAG'].map({'Y': 1, 'N': 0})
df['BOOKING_DOMESTIC_FLAG'] = df['BOOKING_DOMESTIC_FLAG'].map({'Y': 1, 'N': 0})

In [ ]:
df[df['if_additional_upgrade'] == 1]

In [ ]:
emd

In [ ]:
siema